In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine


zip_center=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_Kayla_F_F_20190726'

In [2]:
def get_dist(store_lat,store_lng,zip_cd):
    if zip_cd in zip_center.keys():
        dist_return=haversine([store_lat,store_lng],zip_center[zip_cd],miles=True)
    else:
        dist_return=99999

    return dist_return


In [3]:
store_list=pd.read_excel("./F&F Store List 7-27 and 7-28.xlsx",dtype=str,usecols=[0,9])
new_store_list_Kayla=store_list['New Store List'].tolist()
new_store_list_Kayla=[x for x in new_store_list_Kayla if x!="nan"]

original_store_list_Kayla=store_list['Original Store List'].tolist()
original_store_list_Kayla=[x for x in original_store_list_Kayla if x!="nan"]

store_list=new_store_list_Kayla+original_store_list_Kayla
print(len(store_list))
print(len(set(store_list)))


494
494


In [4]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/output_by_store_zip_sales_cum_60Plus_only_JL_2019-07-26.csv",
                               dtype=str,usecols=['location_id','customer_zip_code','trans_flag']).rename(columns={"customer_zip_code":"zip_cd"})

# store_level_zips['trans']=store_level_zips['trans'].astype(float).astype(int)
store_level_zips['zip_cd']=store_level_zips['zip_cd'].fillna("00000")

store_level_zips=store_level_zips[store_level_zips['trans_flag'].isin(['P', 'S'])]
# store_level_zips['transaction_dt_nunique']=store_level_zips['transaction_dt_nunique'].astype(float).astype(int)
# store_level_zips=store_level_zips[store_level_zips['transaction_dt_nunique']>=60]
store_level_zips.shape

(29386, 3)

In [5]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",
                            sep="|",dtype=str)
df_store_list=df_store_list[['location_id','latitude_meas','longitude_meas',"zip_cd"]].rename(columns={"zip_cd":"store_zip"})
df_store_list['store_zip']=df_store_list['store_zip'].apply(lambda x: x.split("-")[0].zfill(5))

df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list=df_store_list[df_store_list['location_id'].isin(store_list)]
df_store_list.shape

(494, 4)

In [6]:
new_store=[x for x in store_list if x not in df_store_list['location_id'].tolist()]
new_store

[]

In [7]:
df_store_list=pd.merge(df_store_list,store_level_zips,on="location_id",how="left")
df_store_list.shape

(7576, 6)

In [8]:
df_store_list_old=df_store_list[pd.notnull(df_store_list['trans_flag'])]
df_store_list_new=df_store_list[pd.isnull(df_store_list['trans_flag'])]
df_store_list_new.shape

(6, 6)

In [9]:
df_store_list_old['dist_miles']=df_store_list_old.apply(lambda df: get_dist(df['latitude_meas'],df['longitude_meas'],df['zip_cd']),axis=1)
df_store_list_old=df_store_list_old[(df_store_list_old['dist_miles']<=20) | (df_store_list_old['trans_flag']=="P")]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_store_list_old.head(2)

,location_id,latitude_meas,longitude_meas,store_zip,zip_cd,trans_flag,dist_miles
0,375,34.161278,-86.836922,35055,35055,P,1.696242
1,375,34.161278,-86.836922,35055,35057,P,6.392634


In [11]:
del df_store_list_new['zip_cd']
del df_store_list_new['trans_flag']
df_store_list_new

,location_id,latitude_meas,longitude_meas,store_zip
1596,1918,38.563259,-121.380165,95826
6078,4697,30.128432,-94.157048,77708
7261,4683,37.955826,-122.333660,94806
7268,4694,32.677142,-97.413888,76132
7269,4699,34.086000,-118.150738,91803
7575,5385,36.748185,-95.931540,74006


In [12]:
df_store_list_new=df_store_list_new.reset_index()
del df_store_list_new['index']

all_zips_in_10=pd.DataFrame()
for ind,row in df_store_list_new.iterrows():
    store_numer=row['location_id']
    store_center=[row['latitude_meas'],row['longitude_meas']]
    
    for zip_cd in zip_center.keys():
        dist=haversine(store_center,zip_center[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":store_numer,"zip_cd":zip_cd,"trans_flag":"zips_in_10","dist_miles":dist},index=[0])
            all_zips_in_10=all_zips_in_10.append(df)

df_store_list_new=pd.merge(df_store_list_new,all_zips_in_10,on="location_id",how="left")

In [13]:
output=df_store_list_new.append(df_store_list_old)

output=output[df_store_list_old.columns.tolist()]

In [14]:
output['store_type']=np.where(output['location_id'].isin(new_store_list_Kayla),"New Store","Original Store")
output=output[output['zip_cd'].isin(zip_center.keys())]
print(output['location_id'].nunique())
print(output.shape)

494
(6722, 8)


In [15]:
df_unique_zip=output[['zip_cd','trans_flag']].sort_values(['trans_flag','zip_cd'])

In [16]:
writer=pd.ExcelWriter("./BL_zips_494_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"df_unique_zip",index=False)
output.to_excel(writer,"zip_by_store",index=False)
writer.save()

In [17]:
'''
store_level_zips_update_60Plus_Only=pd.DataFrame()

for store,df_store in store_level_zips.groupby("location_id"):
    total_valid_sales=df_store['sales'].sum()
    total_valid_trans=df_store['trans'].sum()
    
    df_store=df_store.sort_values("sales",ascending=False)
    df_store['cumulative_sales']=df_store['sales'].cumsum()
    df_store['cumu_pctg_sales']=df_store['cumulative_sales']/total_valid_sales
    df_store=df_store.reset_index()
    del df_store['index']
    
    df_store_1st=df_store.iloc[0,].to_frame().T
    df_store_1st['revenue_flag']="P"
    
    df_store_others=df_store.iloc[1:,]
    df_store_others['revenue_flag']=np.where(df_store_others['cumu_pctg_sales']>=0.8,"T",
                                  np.where(df_store_others['cumu_pctg_sales']>=0.6,"S","P"))
    df_store=df_store_1st.append(df_store_others)
    ######### 
    df_store=df_store.sort_values("trans",ascending=False)
    df_store['cumulative_trans']=df_store['trans'].cumsum()
    df_store['cumu_pctg_trans']=df_store['cumulative_trans']/total_valid_trans
    df_store=df_store.reset_index()
    del df_store['index']
    
    df_store_1st=df_store.iloc[0,].to_frame().T
    df_store_1st['trans_flag']="P"
    
    df_store_others=df_store.iloc[1:,]
    df_store_others['trans_flag']=np.where(df_store_others['cumu_pctg_trans']>=0.8,"T",
                                  np.where(df_store_others['cumu_pctg_trans']>=0.6,"S","P"))
    df_store=df_store_1st.append(df_store_others)

    store_level_zips_update_60Plus_Only=store_level_zips_update_60Plus_Only.append(df_store)
    
store_level_zips_update_60Plus_Only.to_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/output_by_store_zip_sales_cum_60Plus_only_JL_2019-07-26.csv",index=False)

'''

'\nstore_level_zips_update_60Plus_Only=pd.DataFrame()\n\nfor store,df_store in store_level_zips.groupby("location_id"):\n    total_valid_sales=df_store[\'sales\'].sum()\n    total_valid_trans=df_store[\'trans\'].sum()\n    \n    df_store=df_store.sort_values("sales",ascending=False)\n    df_store[\'cumulative_sales\']=df_store[\'sales\'].cumsum()\n    df_store[\'cumu_pctg_sales\']=df_store[\'cumulative_sales\']/total_valid_sales\n    df_store=df_store.reset_index()\n    del df_store[\'index\']\n    \n    df_store_1st=df_store.iloc[0,].to_frame().T\n    df_store_1st[\'revenue_flag\']="P"\n    \n    df_store_others=df_store.iloc[1:,]\n    df_store_others[\'revenue_flag\']=np.where(df_store_others[\'cumu_pctg_sales\']>=0.8,"T",\n                                  np.where(df_store_others[\'cumu_pctg_sales\']>=0.6,"S","P"))\n    df_store=df_store_1st.append(df_store_others)\n    ######### \n    df_store=df_store.sort_values("trans",ascending=False)\n    df_store[\'cumulative_trans\']=df_sto